In [60]:
%matplotlib widget
from lib import get_tournament_list, get_season_list, ShotDistribution
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [66]:
tourns = get_tournament_list()
seasons = get_season_list()
strokes=[1, 2, 3, 4, 5, 'all', 'drive', 'approach']
scores=[None, 'sub', 'par', 'over', 1, 2, 3, 4, 5, 6, 7, 8, 'all']
holes=[i+1 for i in range(18)]
rounds=[1, 2, 3, 4, 'all']
plt.ioff()
fig = plt.figure(figsize=(6, 6), tight_layout=True)
ax = fig.add_subplot(111)
fig.canvas.toolbar_visible = True
fig.canvas.header_visible = False # Hide the Figure name at the top of the figure
fig.canvas.footer_visible = False
fig.canvas.resizable = True
fig.canvas.capture_scroll = False
plt.ion()

fig = go.Figure()
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        },
    ],
    mapbox=dict(
        center=go.layout.mapbox.Center(
        lat=33.64,
        lon=-111.912,
        ),
        zoom=16,
    ),
    margin={"r":0,"t":0,"l":0,"b":0},
)

s = ShotDistribution()

In [67]:
def get_tourn_data(Tournament, Season):
    global s
    try:
        if Season == 'all':
            s.generate_distribution(Tournament)
        else:
            s.generate_distribution(Tournament, Season)    
    except Exception as e:
        print(e)
        
#     return (Tournament, Season, s)

tourn_data = interactive(get_tourn_data, Tournament=tourns, Season=seasons)

In [81]:
daily_out = widgets.Output(layout = {
            'width': '100%',
            'height': '600px',
            'border': '1px solid black'
        })

def get_hole_data(Hole, Stroke, Round, Score1, Score2, Score3, Include_Shots, Include_Spray):
    global ax
    global fig
#     ax.cla()
    
    with daily_out:
        fig.data = []
        try:
            for score_val in [Score1, Score2, Score3]:
                if Include_Shots:
                    fig = s.plot_hole(fig, Hole, Stroke, score_val, Round)

                if score_val is None:
                    continue

                fig = s.plot_hole_distribution(fig, Hole, Stroke, score_val, Round)
                if Include_Spray:
                    fig = s.plot_shot_distribution(fig, Hole, Stroke, score_val, Round) 

        except Exception as e:
            print(e)
        fig.layout.mapbox.center["lat"] = s.as_df(Hole).lat.mean()
        fig.layout.mapbox.center["lon"] = s.as_df(Hole).lon.mean()
        
        daily_out.clear_output()
        fig.show()
#     ax.grid()
#     ax.axis('equal')
#     ax.layout('tight')
#     return ax

hole=widgets.Dropdown(
    options=holes,
    description='Hole:',
    disabled=False,
)

score1=widgets.Dropdown(
    options=scores,
    description='Score 1:',
    disabled=False,        
    value=None,
)

score2=widgets.Dropdown(
    options=scores,
    description='Score 2:',
    disabled=False,
    value=None,
)

score3=widgets.Dropdown(
    options=scores,
    description='Score 3:',
    disabled=True,
    value=None,
)
stroke=widgets.Dropdown(
    options=strokes,
    description='Stroke:',
    disabled=False,
)

t_round=widgets.Dropdown(
    options=rounds,
    description='Round:',
    disabled=False,
)

inc_hole = widgets.Checkbox(
    value=True,
    description='Include Hole',
    disabled=False
)

inc_spread = widgets.Checkbox(
    value=False,
    description='Include Spread',
    disabled=False
)

hole_data = widgets.interactive_output(
    get_hole_data, 
    {
        "Hole": hole, 
        "Stroke": stroke, 
        "Round": t_round, 
        "Score1": score1, 
        "Score2": score2, 
        "Score3": score3, 
        "Include_Shots": inc_hole,
        "Include_Spray": inc_spread,    
    }
)

input_boxes = widgets.VBox([hole, stroke, t_round, score1, score2, score3, inc_hole, inc_spread]) 
# output = hole_data.children[-1]
# output.layout.height = '1000px'

### Tournament

In [83]:
display(widgets.HBox(tourn_data.children[:-1]))

### Data

In [84]:
display(widgets.HBox([daily_out, input_boxes]))